In [1]:
# !rm -rf ./wide_resnet_poly_2ndtrain

In [2]:
%config IPCompleter.greedy=True
import numpy as np
import tensorflow as tf
import time

In [3]:
%run ./datagen.py
datagen, (x_train, y_train), (x_test, y_test) = data_preparation()

In [4]:
save_dir = './wide_resnet_poly_10_2ndtrain/'
batch_size = 100
iterations = x_train.shape[0] // batch_size
epochs = 1000
old_acc = 0
start_lr = 1e-4
end_lr = 1e-5
decay_rate = (end_lr / start_lr) ** (1 / epochs)
k = 10
# regularizer = tf.contrib.layers.l2_regularizer(scale=1e-4)
# initializer=tf.initializers.he_normal()

In [5]:
def act(x, i):
    
    # ax^2 + bx + c
        
    if i == 1:
        first_coef = tf.Variable(0.31931543, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.49708712, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.12422738, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)
        
    elif i == 2:
        first_coef = tf.Variable(0.14951853, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.49850001, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.26582837, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)
        
    elif i == 3:
        first_coef = tf.Variable(0.16431344, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.48894203, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.23802768, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    elif i == 4:
        first_coef = tf.Variable(0.14941755, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.48750084, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.26098495, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    elif i == 5:
        first_coef = tf.Variable(0.10792727, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.48113492, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.35588625, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    else:
        first_coef = tf.Variable(0.07860769, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.49907132, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.50563614, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    return y

In [6]:
# resnet layer
def res_layer(inputs, filter_num, filter_size, stride, is_train, act_num, 
              batch_norm=True, activation=True):

    x = inputs

    if batch_norm:
        x = tf.layers.batch_normalization(x, training=is_train)
    if activation:
        x = act(x, act_num)
    x = tf.layers.conv2d(inputs=x, filters=filter_num, 
                         kernel_size=filter_size, strides=stride, padding='same')
        
    return x

In [7]:
def wide_resnet(inputs, k, is_train):

    with tf.variable_scope("1st_Conv"):
        x = tf.layers.conv2d(inputs=inputs, filters=16, 
                             kernel_size=3, strides=1, padding='same')
        x = tf.layers.batch_normalization(x, training=is_train)
        x = tf.nn.relu(x)
        tf.summary.histogram('activation', x)

    # Res Blocks
    a = [1, 1, 1]
    act_num = 0

    for stack in range(len(a)):
        for block in range(a[stack]):

            with tf.variable_scope('ResBlock_%d_%d' % (stack+1, block+1)):

                batch_norm = True
                activation = True
                stride = 1
                filter_num = 16*k*(2**stack)
                if stack == 0:
                    if block == 0:
                        batch_norm = False
                        activation = False
                else:  
                    if block == 0:
                        stride = 2

                shortcut = x
                with tf.variable_scope('conv1'):
                    x = res_layer(x, filter_num, 3, stride, is_train, act_num, 
                                  batch_norm=batch_norm, activation=activation)
                    act_num += 1
                    
                x = tf.layers.dropout(x, 0.1)
                
                with tf.variable_scope('conv2'):
                    x = res_layer(x, filter_num, 3, 1, is_train, act_num)
                    act_num += 1
                
                with tf.variable_scope('x_plus_shortcut'):
                    if block == 0:
                        shortcut = tf.layers.conv2d(inputs=shortcut, filters=filter_num, 
                                                    kernel_size=1, strides=stride, 
                                                    padding='same')
                    x = x + shortcut
                tf.summary.histogram('output', x)
    
    with tf.variable_scope("AfterResBlock"):
        x = tf.layers.batch_normalization(x, training=is_train)
        x = act(x, act_num)
        x = tf.layers.average_pooling2d(x, pool_size=8, strides=8, 
                                        padding='SAME', name='ave_pool')
        tf.summary.histogram('output', x)
        
    with tf.variable_scope("Flatten"):
        x = tf.transpose(x, perm=[0, 3, 1, 2])
        x = tf.layers.flatten(x)
    
    # crrent x.shape = (?, 256)
    with tf.variable_scope("Prediction"):
        pred = tf.layers.dense(x, units=10)
        tf.summary.histogram('output', x)
        
    return pred

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

tf.reset_default_graph()

with tf.device('/GPU:7'):

    inputs = tf.placeholder(tf.float32, [None, 32, 32, 3], name='input')
    outputs = tf.placeholder(tf.float32, [None, 10], name='output')
    is_train = tf.placeholder(tf.bool, name='is_train')

    global_step = tf.Variable(0, trainable=False)

    l_r = tf.train.exponential_decay(
        start_lr, global_step, iterations, decay_rate, staircase=True)
    tf.summary.scalar('learning_rate', l_r)

    opt = tf.train.AdamOptimizer(learning_rate=l_r)

    pred = wide_resnet(inputs, k, is_train)
    
#     l2_loss = tf.losses.get_regularization_loss()
    loss = tf.losses.softmax_cross_entropy(outputs, pred)
    
    grads = opt.compute_gradients(loss)

    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(outputs, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.control_dependencies(update_ops):
        train_op = opt.apply_gradients(grads, global_step=global_step)

    restore_vars = []

    for var in tf.global_variables()[1:51+18]:
        if '_coef:0' not in var.name:
            restore_vars.append(var)
    saver = tf.train.Saver(restore_vars)
    
    new_saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)

    def add_hist(train_vars):
        for i in train_vars:
            name = i.name.split(":")[0] + '/value'
            value = i.value()
            tf.summary.histogram(name, value)

    add_hist(tf.trainable_variables())
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    merged = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [9]:
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)

print('*****************2nd training stage*****************')
sess.run(tf.global_variables_initializer())
saver.restore(sess, './wide_resnet_poly_10/'+'cifar10.ckpt-211500')
train_writer = tf.summary.FileWriter(save_dir+'train', sess.graph)

for m in range(epochs):
        start = time.time()
        batch_gen = datagen.flow(
            x_train, y_train, batch_size=batch_size)

        for i in range(iterations):
            x_batch, y_batch = next(batch_gen)
            _, loss_train = sess.run([train_op, loss], 
                                     {inputs: x_batch, outputs: y_batch, is_train: True})
        
        summary = sess.run(merged, {inputs: x_batch, outputs: y_batch, is_train: False})
        train_writer.add_summary(summary, m*iterations + i + 1)

        val_accs = []
        for i in range(5000//(batch_size*5)):
            val_acc = sess.run(accuracy, {inputs: x_test[i*batch_size*5: (i+1)*batch_size*5],
                                          outputs: y_test[i*batch_size*5: (i+1)*batch_size*5],
                                          is_train: False})
            val_accs.append(val_acc)

        if np.mean(val_accs) > old_acc:
            old_acc = np.mean(val_accs)
            new_saver.save(sess, save_dir+'cifar10.ckpt', global_step=global_step)

        end = time.time()
        print('Epoch: {}'.format(m + 1),
              'Train_loss: {:.3f}'.format(loss_train),
              'Val_acc: {:.3f}'.format(np.mean(val_accs)),
              'Time consumed: {:.4f} s'.format(end - start))

print('*****************Training End!*****************')

*****************2nd training stage*****************
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./wide_resnet_poly_10/cifar10.ckpt-211500
Epoch: 1 Train_loss: 2.001 Val_acc: 0.279 Time consumed: 67.8830 s
Epoch: 2 Train_loss: 1.850 Val_acc: 0.387 Time consumed: 64.3883 s
Epoch: 3 Train_loss: 1.072 Val_acc: 0.431 Time consumed: 63.3540 s
Epoch: 4 Train_loss: 0.432 Val_acc: 0.545 Time consumed: 63.1870 s
Epoch: 5 Train_loss: 0.288 Val_acc: 0.754 Time consumed: 63.2170 s
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: 6 Train_loss: 0.271 Val_acc: 0.854 Time consumed: 63.4492 s
Epoch: 7 Train_loss: 0.157 Val_acc: 0.861 Time consumed: 63.5441 s
Epoch: 8 Train_loss: 0.068 Val_acc: 0.853 Time consumed: 63.1686 s
Epoch: 9 Train_loss: 0.180 Val_acc: 0.866 Time consumed: 63.6315 s
Epoch: 10 Train_loss: 0.170 Val_acc: 0.875 Time consumed: 63.7632 s
Epoch: 11 Train_loss:

Epoch: 111 Train_loss: 0.000 Val_acc: 0.905 Time consumed: 61.9794 s
Epoch: 112 Train_loss: 0.006 Val_acc: 0.909 Time consumed: 62.1343 s
Epoch: 113 Train_loss: 0.006 Val_acc: 0.902 Time consumed: 61.9018 s
Epoch: 114 Train_loss: 0.008 Val_acc: 0.902 Time consumed: 61.9412 s
Epoch: 115 Train_loss: 0.001 Val_acc: 0.906 Time consumed: 62.1251 s
Epoch: 116 Train_loss: 0.017 Val_acc: 0.909 Time consumed: 62.0887 s
Epoch: 117 Train_loss: 0.003 Val_acc: 0.907 Time consumed: 61.9393 s
Epoch: 118 Train_loss: 0.002 Val_acc: 0.910 Time consumed: 62.2007 s
Epoch: 119 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 61.9482 s
Epoch: 120 Train_loss: 0.002 Val_acc: 0.906 Time consumed: 62.1415 s
Epoch: 121 Train_loss: 0.001 Val_acc: 0.905 Time consumed: 62.3846 s
Epoch: 122 Train_loss: 0.004 Val_acc: 0.908 Time consumed: 62.0383 s
Epoch: 123 Train_loss: 0.002 Val_acc: 0.906 Time consumed: 61.9782 s
Epoch: 124 Train_loss: 0.003 Val_acc: 0.906 Time consumed: 61.7918 s
Epoch: 125 Train_loss: 0.003 Val_a

Epoch: 230 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.1516 s
Epoch: 231 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.4062 s
Epoch: 232 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 62.7761 s
Epoch: 233 Train_loss: 0.001 Val_acc: 0.907 Time consumed: 62.7712 s
Epoch: 234 Train_loss: 0.001 Val_acc: 0.909 Time consumed: 62.2235 s
Epoch: 235 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 61.9810 s
Epoch: 236 Train_loss: 0.001 Val_acc: 0.912 Time consumed: 63.4234 s
Epoch: 237 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.4496 s
Epoch: 238 Train_loss: 0.001 Val_acc: 0.905 Time consumed: 62.2725 s
Epoch: 239 Train_loss: 0.000 Val_acc: 0.907 Time consumed: 62.0184 s
Epoch: 240 Train_loss: 0.005 Val_acc: 0.908 Time consumed: 62.2869 s
Epoch: 241 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.2984 s
Epoch: 242 Train_loss: 0.000 Val_acc: 0.906 Time consumed: 62.1311 s
Epoch: 243 Train_loss: 0.001 Val_acc: 0.908 Time consumed: 62.3755 s
Epoch: 244 Train_loss: 0.018 Val_a

Epoch: 349 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 62.7284 s
Epoch: 350 Train_loss: 0.000 Val_acc: 0.906 Time consumed: 62.5234 s
Epoch: 351 Train_loss: 0.001 Val_acc: 0.910 Time consumed: 62.9161 s
Epoch: 352 Train_loss: 0.003 Val_acc: 0.910 Time consumed: 62.7115 s
Epoch: 353 Train_loss: 0.003 Val_acc: 0.911 Time consumed: 62.9058 s
Epoch: 354 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 63.9215 s
Epoch: 355 Train_loss: 0.013 Val_acc: 0.909 Time consumed: 62.5409 s
Epoch: 356 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.3279 s
Epoch: 357 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 62.6132 s
Epoch: 358 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 63.1380 s
Epoch: 359 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.6350 s
Epoch: 360 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 62.7191 s
Epoch: 361 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 63.3338 s
Epoch: 362 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 63.1659 s
Epoch: 363 Train_loss: 0.000 Val_a

Epoch: 468 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.6642 s
Epoch: 469 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.6847 s
Epoch: 470 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 62.9075 s
Epoch: 471 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 62.5053 s
Epoch: 472 Train_loss: 0.001 Val_acc: 0.910 Time consumed: 62.6731 s
Epoch: 473 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 63.4218 s
Epoch: 474 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 63.0655 s
Epoch: 475 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 62.8470 s
Epoch: 476 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 63.0828 s
Epoch: 477 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.7431 s
Epoch: 478 Train_loss: 0.000 Val_acc: 0.908 Time consumed: 62.9201 s
Epoch: 479 Train_loss: 0.006 Val_acc: 0.909 Time consumed: 62.6877 s
Epoch: 480 Train_loss: 0.001 Val_acc: 0.908 Time consumed: 63.0363 s
Epoch: 481 Train_loss: 0.001 Val_acc: 0.908 Time consumed: 63.2285 s
Epoch: 482 Train_loss: 0.022 Val_a

Epoch: 587 Train_loss: 0.001 Val_acc: 0.912 Time consumed: 62.5947 s
Epoch: 588 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 62.7548 s
Epoch: 589 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 63.2146 s
Epoch: 590 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.5542 s
Epoch: 591 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 64.7311 s
Epoch: 592 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 62.3505 s
Epoch: 593 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 62.5014 s
Epoch: 594 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 62.2224 s
Epoch: 595 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.3325 s
Epoch: 596 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.2355 s
Epoch: 597 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.8720 s
Epoch: 598 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 62.5598 s
Epoch: 599 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.9323 s
Epoch: 600 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 62.7926 s
Epoch: 601 Train_loss: 0.000 Val_a

Epoch: 706 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.4202 s
Epoch: 707 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 62.6634 s
Epoch: 708 Train_loss: 0.002 Val_acc: 0.911 Time consumed: 62.1229 s
Epoch: 709 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 61.5345 s
Epoch: 710 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 61.7699 s
Epoch: 711 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.3927 s
Epoch: 712 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 61.3302 s
Epoch: 713 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.4203 s
Epoch: 714 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 61.7393 s
Epoch: 715 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 61.9071 s
Epoch: 716 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 62.4320 s
Epoch: 717 Train_loss: 0.001 Val_acc: 0.913 Time consumed: 61.1957 s
Epoch: 718 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.1733 s
Epoch: 719 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 61.1996 s
Epoch: 720 Train_loss: 0.000 Val_a

Epoch: 825 Train_loss: 0.000 Val_acc: 0.911 Time consumed: 61.1912 s
Epoch: 826 Train_loss: 0.000 Val_acc: 0.909 Time consumed: 61.5025 s
Epoch: 827 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 61.4334 s
Epoch: 828 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.3942 s
Epoch: 829 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.4538 s
Epoch: 830 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.7442 s
Epoch: 831 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.4137 s
Epoch: 832 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.5374 s
Epoch: 833 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.4679 s
Epoch: 834 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.1192 s
Epoch: 835 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.5534 s
Epoch: 836 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.7991 s
Epoch: 837 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.5981 s
Epoch: 838 Train_loss: 0.001 Val_acc: 0.912 Time consumed: 61.4910 s
Epoch: 839 Train_loss: 0.000 Val_a

Epoch: 944 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 62.2102 s
Epoch: 945 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.7975 s
Epoch: 946 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.8009 s
Epoch: 947 Train_loss: 0.000 Val_acc: 0.914 Time consumed: 61.7092 s
Epoch: 948 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.5479 s
Epoch: 949 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.4094 s
Epoch: 950 Train_loss: 0.000 Val_acc: 0.910 Time consumed: 62.6602 s
Epoch: 951 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.1533 s
Epoch: 952 Train_loss: 0.006 Val_acc: 0.911 Time consumed: 61.4823 s
Epoch: 953 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.1136 s
Epoch: 954 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.3037 s
Epoch: 955 Train_loss: 0.000 Val_acc: 0.912 Time consumed: 61.5895 s
Epoch: 956 Train_loss: 0.000 Val_acc: 0.913 Time consumed: 61.1614 s
Epoch: 957 Train_loss: 0.000 Val_acc: 0.914 Time consumed: 61.4242 s
Epoch: 958 Train_loss: 0.000 Val_a